In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
import operator
import random
import csv
import numpy
import os
from deap import base, creator, tools, gp, algorithms
from functools import partial
from multiprocessing import Pool

In [ ]:
class RegisterMachine:
    def __init__(self, array=[]):
        self.flag = 0
        self.regs = [0] * 4  # 4 registres : A, D, I, BorneSup
        self.array = array[:]

    def _reset(self, array):
        self.flag = 0
        self.regs = [0] * 4
        self.array = array[:]

    def maxIter(self):
        return len(self.array)

    def movBmaxIter(self):
        self.regs[3] = len(self.array)

    def clrI(self):
        self.regs[2] = 0

    def incI(self):
        self.regs[2] += 1

    def decI(self):
        self.regs[2] -= 1

    def clrA(self):
        self.regs[0] = 0

    def movAD(self):
        self.regs[1] = self.regs[0]

    def movDA(self):
        self.regs[0] = self.regs[1]

    def movIA(self):
        self.regs[0] = self.regs[2]

    def swapAD(self):
        self.regs[0], self.regs[1] = self.regs[1], self.regs[0]

    def movGteA(self):
        if self.regs[1] >= self.regs[0]:
            self.swapAD()

    def readArrayD(self):
        self.regs[1] = self.array[self.regs[2] % len(self.array)]

    def run(self, routine, fitcases):
        self._reset(fitcases)
        try:
            routine()
        except Exception as e:
            self.flag = 1  # Indicate an error
        return self.regs[0] if not self.flag else -999


machine = RegisterMachine()

In [ ]:
def progn(*args):
    for arg in args:
        arg()


def prog2(out1, out2):
    return partial(progn, out1, out2)


def whileLoopCode(code):
    cpt = 0
    previous_values = set()
    while machine.regs[2] < machine.regs[3]:
        # print(f"Iteration {cpt}: regs[2]={machine.regs[2]}, regs[3]={machine.regs[3]}")
        # if cpt > 1000 or machine.regs[2] in previous_values:
        if machine.regs[2] in previous_values:
            # print(f"Loop detected as infinite or exceeded max iterations: {cpt}, regs[2]: {machine.regs[2]}, regs[3]: {machine.regs[3]}")
            break
        previous_values.add(machine.regs[2])
        code()
        cpt += 1


def whileLoop(code):
    return partial(whileLoopCode, code)

In [ ]:
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(prog2, 2)
pset.addPrimitive(whileLoop, 1)

# Terminals restreints aux opÃ©rations essentielles
pset.addTerminal(machine.movBmaxIter)
pset.addTerminal(machine.incI)
pset.addTerminal(machine.clrI)
pset.addTerminal(machine.swapAD)
pset.addTerminal(machine.movGteA)
pset.addTerminal(machine.readArrayD)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# Limiter la profondeur des arbres
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def genFitCase(mult_size=1, bound=20):
    seq = []
    for i in range(10 * mult_size):
        nbr = random.randint(-bound, bound)
        seq.append(nbr)
    return (max(seq), seq, sorted(seq, reverse=True))

'''
def genFitCase(mult_size=1, bound=20, min_size=5, max_size=15, add_noise=True, noise_prob=0.2):
    size = random.randint(min_size, max_size) * mult_size
    seq = [random.randint(-bound, bound) for _ in range(size)]

    # Ajouter du bruit avec une certaine probabilitÃ©
    if add_noise and random.random() < noise_prob:
        seq[random.randint(0, size - 1)] = random.choice(
            [random.randint(2 * bound, 3 * bound), random.randint(-3 * bound, -2 * bound)])

    return (max(seq), seq, sorted(seq, reverse=True))
'''

'\ndef genFitCase(mult_size=1, bound=20, min_size=5, max_size=15, add_noise=True, noise_prob=0.2):\n    size = random.randint(min_size, max_size) * mult_size\n    seq = [random.randint(-bound, bound) for _ in range(size)]\n\n    # Ajouter du bruit avec une certaine probabilitÃ©\n    if add_noise and random.random() < noise_prob:\n        seq[random.randint(0, size - 1)] = random.choice(\n            [random.randint(2 * bound, 3 * bound), random.randint(-3 * bound, -2 * bound)])\n\n    return (max(seq), seq, sorted(seq, reverse=True))\n'

In [ ]:
# cases = [genFitCase(random.randint(1, 5), 100) for _ in range(50)]  # Diversifier les tailles
cases = [genFitCase(2, 100) for _ in range(50)]

In [ ]:
def evalMaxList(individual, cases):
    routine = gp.compile(individual, pset)
    error = 0
    complexity = len(individual)
    try:
        for case in cases:
            answer = machine.run(routine, case[1])
            if answer in case[2]:
                error += case[2].index(answer)
            else:
                error += len(case[2])
    except RuntimeError:
        return float('inf'),  # Penalize infinite loops heavily

    # Combiner l'erreur avec la complexitÃ©
    return (error / len(cases)) + 0.01 * complexity,

In [ ]:
toolbox.register("evaluate", evalMaxList, cases=cases)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=3)  # RÃ©duire la profondeur de mutation
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.register("map", Pool().map)

In [ ]:
def deap_to_java(individual):
    java_code = """
public class MaxArray {

    public int maxArray(int[] array) {
        int regA = 0;
        int regD = 0;
        int regI = 0;
        int regB = 0;
"""
    translate = {
        'prog2': lambda args: '\n'.join(args),
        'whileLoop': lambda args: f"while (regI != regB) {{\n{args[0]}\n}}",
        'movBmaxIter': "regB = array.length;",
        'clrI': "regI = 0;",
        'incI': "regI++;",
        'swapAD': "int temp = regA; regA = regD; regD = temp;",
        'movGteA': "if (regD >= regA) { int temp = regA; regA = regD; regD = temp; }",
        'readArrayD': "regD = array[regI % array.length];",
    }

    def convert_expression(expr):
        return convert_expression_internal(expr, 0)

    def convert_expression_internal(expr, indice):
        current = expr[indice]
        if isinstance(current, gp.Primitive):
            indice += 1
            args = []
            for _ in range(current.arity):
                translation, indice = convert_expression_internal(expr, indice)
                args.append(translation)
            return translate[current.name](args), indice
        elif isinstance(current, gp.Terminal):
            return translate[current.name], indice + 1

    java_instructions, _ = convert_expression(individual) or ("", 0)
    java_code += java_instructions

    java_code += """
        return regA;
    }
}
"""
    return java_code


In [ ]:
def save_individual_as_java(run, gen, ind_num, individual, fitness_value, best=False, best_run=False, csv_writer=None):
    # Convertir l'individu en code Java
    java_code = deap_to_java(individual)

    # Convertir l'individu en chaÃ®ne de caractÃ¨res pour le stocker dans le CSV
    individual_str = str(individual)

    # DÃ©finir le chemin du dossier pour le run et la gÃ©nÃ©ration
    run_folder = f"run{run + 1}"

    if best_run:
        # Si c'est le meilleur individu du run, enregistrer dans le dossier du run
        path = run_folder
        filename = f"best_run{run + 1}.java"
    else:
        # Sinon, enregistrer dans le sous-dossier de la gÃ©nÃ©ration
        gen_folder = f"gen{gen + 1}"
        path = os.path.join(run_folder, gen_folder)
        os.makedirs(path, exist_ok=True)  # CrÃ©er les dossiers si nÃ©cessaire

        # DÃ©finir le nom du fichier
        if best:
            filename = f"best_ind_run{run + 1}_gen{gen + 1}.java"
        elif ind_num is not None:
            filename = f"run{run + 1}_gen{gen + 1}_ind{ind_num + 1}.java"
        else:
            filename = f"individual_run{run + 1}_gen{gen + 1}.java"

    filepath = os.path.join(path, filename)

    # Ã‰crire le code Java dans le fichier
    with open(filepath, "w") as file:
        file.write(java_code)

    # Enregistrer les informations dans le fichier CSV, en ajoutant l'individu avant transformation
    if csv_writer is not None:
        csv_writer.writerow([filename, fitness_value, individual_str])

    print(f"Saved {filepath} with fitness {fitness_value}")

In [ ]:
import json

def save_population_as_trees_json(population, generation, run):
    data = {"generation": generation, "run": run, "individuals": []}
    for ind_num, ind in enumerate(population):
        data["individuals"].append({"id": ind_num + 1, "tree": str(ind)})

    file_name = f"run{run}_gen{generation}_population_trees.json"
    with open(file_name, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Saved population as trees for generation {generation}, run {run} to {file_name}")


In [ ]:
import os
import random
import json
import numpy as np
from deap import tools

# Define save functions
def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

def save_individual(run, gen, ind_id, individual, fitness_value, complexity, parent_ids, seed):
    return {
        "run": run + 1,
        "generation": gen + 1,
        "individual_id": ind_id + 1,
        "individual": str(individual),
        "seed": seed,
        "fitness": fitness_value,
        "complexity": complexity
    }

def save_generation_statistics(run, gen, mean_fitness, std_fitness, mean_complexity, best_fitness):
    return {
        "run": run + 1,
        "generation": gen + 1,
        "mean_fitness": mean_fitness,
        "std_fitness": std_fitness,
        "mean_complexity": mean_complexity,
        "best_fitness": best_fitness
    }

def main():
    NGEN = 50
    CXPB, MUTPB = 0.7, 0.3
    NUM_RUNS = 20
    POP_SIZE = 1000

    # Save the seed for reproducibility
    SEED = NUM_RUNS
    random.seed(SEED)

    # Structure for the overall JSON file
    all_data = {
        "population": [],
        "generations_statistics": [],
        "bests_individuals": []
    }

    # Global ID tracker for individuals
    current_id = 0
    parent_tracker = {}

    for run in range(NUM_RUNS):
        print(f"Run {run + 1}/{NUM_RUNS}")

        pop = toolbox.population(n=POP_SIZE)
        hof = tools.HallOfFame(1)
        stats_fit = tools.Statistics(lambda ind: ind.fitness.values[0])
        stats_size = tools.Statistics(len)
        mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
        mstats.register("std", np.std)
        mstats.register("min", np.min)
        mstats.register("max", np.max)

        # Assign initial IDs to the population
        for ind in pop:
            parent_tracker[current_id] = {"parents": None, "individual": str(ind)}
            ind.id = current_id
            current_id += 1

        for gen in range(NGEN):
            print(f"Generation {gen + 1}/{NGEN}")

            fitnesses = [toolbox.evaluate(ind) for ind in pop]
            for ind, fit in zip(pop, fitnesses):
                ind.fitness.values = fit

            fitness_vals = [ind.fitness.values[0] for ind in pop]
            complexities = [len(ind) for ind in pop]

            # Record population data
            for ind_id, ind in enumerate(pop):
                individual_data = save_individual(
                    run, gen, ind_id, ind, ind.fitness.values[0], len(ind),
                    parent_tracker.get(ind.id, {}).get("parents"), SEED
                )
                all_data["population"].append(individual_data)

            # Record generation statistics
            generation_stats = save_generation_statistics(
                run, gen, np.mean(fitness_vals), np.std(fitness_vals), np.mean(complexities), min(fitness_vals)
            )
            all_data["generations_statistics"].append(generation_stats)

            # Record best individual
            best_ind = tools.selBest(pop, 1)[0]
            best_ind_data = save_individual(
                run, gen, -1, best_ind, best_ind.fitness.values[0], len(best_ind),
                parent_tracker.get(best_ind.id, {}).get("parents"), SEED
            )
            all_data["bests_individuals"].append(best_ind_data)

            # Evolve population
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))

            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < CXPB:
                    toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values

                    # Assign new IDs and track parents
                    child1.id = current_id
                    parent_tracker[current_id] = {"parents": [child1.id, child2.id], "individual": str(child1)}
                    current_id += 1

                    child2.id = current_id
                    parent_tracker[current_id] = {"parents": [child1.id, child2.id], "individual": str(child2)}
                    current_id += 1

            for mutant in offspring:
                if random.random() < MUTPB:
                    toolbox.mutate(mutant)
                    del mutant.fitness.values

                    # Assign new ID and track parent
                    mutant.id = current_id
                    parent_tracker[current_id] = {"parents": [mutant.id], "individual": str(mutant)}
                    current_id += 1

            pop[:] = offspring

    # Save all data to a single JSON file
    save_to_json(all_data, "all_runs_data.json")
    print("All data saved to all_runs_data.json")

if __name__ == "__main__":
    main()


Run 1/20
Generation 1/50
Generation 2/50
Generation 3/50
Generation 4/50
Generation 5/50
Generation 6/50
Generation 7/50
Generation 8/50
Generation 9/50
Generation 10/50
Generation 11/50
Generation 12/50
Generation 13/50
Generation 14/50
Generation 15/50
Generation 16/50
Generation 17/50
Generation 18/50
Generation 19/50
Generation 20/50
Generation 21/50
Generation 22/50
Generation 23/50
Generation 24/50
Generation 25/50
Generation 26/50
Generation 27/50
Generation 28/50
Generation 29/50
Generation 30/50
Generation 31/50
Generation 32/50
Generation 33/50
Generation 34/50
Generation 35/50
Generation 36/50
Generation 37/50
Generation 38/50
Generation 39/50
Generation 40/50
Generation 41/50
Generation 42/50
Generation 43/50
Generation 44/50
Generation 45/50
Generation 46/50
Generation 47/50
Generation 48/50
Generation 49/50
Generation 50/50
Run 2/20
Generation 1/50
Generation 2/50
Generation 3/50
Generation 4/50
Generation 5/50
Generation 6/50
Generation 7/50
Generation 8/50
Generation 9/